# Ising Model

In [8]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
%matplotlib tk         #use this for a new window popup when plotting

np.random.seed(35)

## Simple Ising model

In [2]:
def initialstate(N):   
    state = np.random.randint(2, size=(N,N))*2-1
    return state

def mcmove(config, beta,J): #our baseline algorithm (we will expand on this algorithm)
    for i in range(len(config)):
        for j in range(len(config)):
            a = np.random.randint(0, len(config))
            b = np.random.randint(0, len(config))
            spin = config[a,b]
            nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]            
            emn = J*spin*nb
            if emn < 0:
                config[a,b] = -config[a,b]
            else:
                p = np.exp(-emn*beta)
                u = np.random.rand()
                if u < p:
                    config[a,b] = -config[a,b]
    return config

def get_Energy(config,J):
    energy = 0
    for i in range(len(config)):
        for j in range(len(config)):
            s = config[i,j]
            nb = config[(i+1)%N, j] + config[i,(j+1)%N] + config[(i-1)%N, j] + config[i,(j-1)%N]            
            energy += -nb*s*J
    return energy

def get_Mag(config):
    mag = np.sum(config)
    return mag

### Let's look at the behaviors of the lattice simulation as we change the temperature and also the coupling interaction energy

Let's first start with temperature less than Curie temperature.

In [ ]:
### Let's check if the code above works or not...
N = 50                  #grid size
iterations = 1000          #number of mcmc move
temp = 0.001                   #Temperature in Kelvins --- super low temperature

grid = initialstate(N)       #initialize lattice spins

plt.figure(figsize = (7,7)) 
plt.imshow(grid)
plt.title('Initial Conditions')

for i in range(iterations):   #MCMC move
    grid = mcmove(grid,1./temp,2) #config, beta, J
    
plt.figure(figsize = (7,7))
plt.imshow(grid)
plt.title('After '+ str(iterations*iterations*iterations) + ' Iterations')

Then, temperature greater than curie

In [10]:
temp = 5                  #Temperature in Kelvins  #high temperature T >> curie temp

grid = initialstate(N)       #initialize lattice spins

plt.figure(figsize = (7,7)) 
plt.imshow(grid)
plt.title('Initial Conditions')

for i in range(iterations):   #MCMC move
    grid = mcmove(grid,1./temp,2) #config, beta, J
    
plt.figure(figsize = (7,7))
plt.imshow(grid)
plt.title('After '+ str(iterations*iterations*iterations) + ' Iterations')

Text(0.5,1,'After 1000000000 Iterations')

Antiferromagnet J < 0

In [11]:
temp = 0.005                  #Temperature in Kelvins

grid = initialstate(N)       #initialize lattice spins

plt.figure(figsize = (7,7)) 
plt.imshow(grid)
plt.title('Initial Conditions')

for i in range(iterations):   #MCMC move
    grid = mcmove(grid,1./temp,-2) #config, beta, J (antiferromagnet)
    
plt.figure(figsize = (7,7))
plt.imshow(grid)
plt.title('After '+ str(iterations*iterations*iterations) + ' Iterations')

Text(0.5,1,'After 1000000000 Iterations')

In [13]:
temp = 3              #Temperature in Kelvins

grid = initialstate(N)       #initialize lattice spins

plt.figure(figsize = (7,7)) 
plt.imshow(grid)
plt.title('Initial Conditions')

for i in range(iterations):   #MCMC move
    grid = mcmove(grid,1./temp,-2) #config, beta, J (antiferromagnet)
    
plt.figure(figsize = (7,7))
plt.imshow(grid)
plt.title('After '+ str(iterations*iterations*iterations) + ' Iterations')

Text(0.5,1,'After 1000000000 Iterations')

### Moving on the see the observable quantities: energy, magnetization, specific heat, and suspectibility

In [14]:
#let's look at some of the observable quantities...
#careful this takes a while!
nt      = 100        #  number of temperature points
N       = 22         #  size of the lattice, N x N
iterations = 1024       # number of iterations for convergence + calculations

T       = np.linspace(0.5, 5, nt);   #temperature
E,M,C,X = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
n1, n2  = 1.0/(iterations*N*N), 1.0/(iterations*iterations*N*N)

for tt in range(len(T)):
    E1 = 0
    M1 = 0
    E2 = 0
    M2 = 0
    init = initialstate(N)
    iT = 1.0/T[tt]
    iT2 = iT**2
    
    for i in range(iterations):     #loop to convergence first
        grid = mcmove(init,iT,2)
    
    for i in range(iterations):     #start calculating the quantities when converged
        grid = mcmove(grid,iT,2)
        en = get_Energy(grid,2)
        mag = get_Mag(grid)
        
        
        E1 = E1 + en            #added to find the "average" quantities for each MC move.
        M1 = M1 + mag
        M2 = M2 + mag**2
        E2 = E2 + en**2
        
    E[tt] = n1*E1
    M[tt] = n1*M1
    C[tt] = (n1*E2 - n2*E1**2)*iT2
    X[tt] = (n1*M2 - n2*M1**2)*iT

In [20]:
fig,ax=plt.subplots(2,2,figsize=(15,12),squeeze = False)

ax[0][0].plot(T,E/4.,'o')
ax[0][0].grid()
ax[0][0].set_xlabel('Temperature',fontsize = 12)
ax[0][0].set_ylabel('Energy',fontsize = 12)

ax[0][1].plot(T,abs(M),'o')
ax[0][1].grid()
ax[0][1].set_xlabel('Temperature', fontsize = 12)
ax[0][1].set_ylabel('Magnetization',fontsize = 12)

ax[1][0].plot(T,C,'o')
ax[1][0].grid()
ax[1][0].set_xlabel('Temperature',fontsize =12)
ax[1][0].set_ylabel('Specific Heat',fontsize =12)

ax[1][1].plot(T,X,'o')
ax[1][1].grid()
ax[1][1].set_xlabel('Temperature',fontsize = 12)
ax[1][1].set_ylabel('Magnetic Suspectibility',fontsize =12)

Text(0,0.5,'Magnetic Suspectibility')

#### Osgner's Exact Solutions for critical temperature and spontenous magnetization are:


$$\sinh(\frac{2J_{1}}{kT_{c}}) \sinh(\frac{2J_{2}}{kT_{c}}) = 1$$
$$M = (1- [\sinh (2\beta J_{1}) \sinh(2\beta J_{2})^{-2}])^{\frac{1}{8}}$$
where $J_{1}$ and $J_{2}$ are the horizontal and vertical coupling interaction strength, $k$ is Boltzmann constant (we use 1). Since our condition is $J_{1} = J_{2} = J$, then the formula can be simplified as

$$T_{c} = \frac{2J}{k\ln(1+\sqrt{2})}$$
$$m = (1- [\sinh(2 \beta J)]^{-4})^{\frac{1}8}$$
for $T < T_{c}$ and 0 for $T > T_{c}$.

Let's plot it aganst the simulation:

In [22]:
J = 1.
crit_temp = (2*J)/(1*np.log(1+np.sqrt(2)))

def exact_mag(J,temp):
    mag = list()
    for i in temp:
        beta = 1./i
        mag_val = (1-np.sinh(2*beta*J)**-4)**(1./8)
        mag.append(mag_val)
    return mag


temp = np.linspace(0.5,5,nt)
plt.plot(temp,abs(M),'o',label = 'Simulation')


temp_exact = np.linspace(0.5,crit_temp,nt)
mag_ex = exact_mag(J,temp_exact)
plt.plot(temp_exact,mag_ex,color = 'red',)
plt.plot(np.linspace(crit_temp,5,nt),np.zeros(len(np.linspace(crit_temp,5,nt))),color = 'red',label = 'Onsager\'s')
plt.grid()
plt.xlabel('Temperature')
plt.ylabel('Magnitization')
plt.legend()

## Introducing External Magnetic Field (scalar)

In [6]:
def mcmove_ext(config,beta,J,ext):
    N = len(config)
    for i in range(N):
        for j in range(N):
            a = np.random.randint(0, N)
            b = np.random.randint(0, N)
            spin = config[a,b]
            nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]       
            emn = J*spin*nb - ext*spin   #magnetic field is here
            if emn < 0:
                config[a,b] = -config[a,b]
            else:
                p = np.exp(-emn*beta)
                u = np.random.rand()
                if u < p:
                    config[a,b] = -config[a,b]
    return config

In [7]:
temp = 3
iterations = 1000

grid_ext = initialstate(50)
plt.figure(figsize = (7,7))
plt.imshow(grid_ext)
plt.title('Initital Conditions')

for i in range(iterations):
    grid_ext = mcmove_ext(grid_ext,1./temp,2,6) #config, beta, J, scalar external field.
    
    
plt.figure(figsize=(7,7))
plt.imshow(grid_ext)
plt.title('After '+ str(iterations*iterations*iterations)+' Simulation')

Text(0.5,1,'After 1000000000 Simulation')

### External Magnetic field (vectorized)

We can create magnetic field to apply it on to certain regions of the lattice to maintain its spin.

In [8]:
def mcmove_vext(config,beta,J,field):
    N = len(config)
    for i in range(N):
        for j in range(N):
            a = np.random.randint(0, N)
            b = np.random.randint(0, N)
            field_strength = field[a,b]
            spin = config[a,b]
            nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]       
            emn = J*spin*nb - field_strength*spin
            if emn < 0:
                config[a,b] = -config[a,b]
            else:
                p = np.exp(-emn*beta)
                u = np.random.rand()
                if u < p:
                    config[a,b] = -config[a,b]
    return config

In [9]:
temp = 4
grid_size = 50
grid_vext = initialstate(grid_size)

magf1 = np.ones((grid_size,grid_size))

magf1[:int(grid_size/2.),:int(grid_size/2.)]= 1
magf1[int(grid_size/2.):,:int(grid_size/2.)]= 20
magf1[int(grid_size/2.):,int(grid_size/2.):]= 1
magf1[:int(grid_size/2.),int(grid_size/2.):] = 20

plt.title('External Magnetic Field')
plt.imshow(magf1)
plt.colorbar()


plt.figure(figsize = (7,7))
plt.imshow(grid_vext)
plt.title('Initital Conditions')
for i in range(iterations):
    grid_vext = mcmove_vext(grid_vext,1./temp,2,magf1) #config, beta, J, scalar external field.
    
    
plt.figure(figsize=(7,7))
plt.imshow(grid_vext)
plt.title('After '+ str(iterations*iterations*iterations)+ ' Simulation')

Text(0.5,1,'After 1000000000 Simulation')

We can get creative with our magnetic field!

## Simple model Simulation

In [10]:
def mcmove(config, beta,J):
    N = len(config)
    a = np.random.randint(0, len(config))
    b = np.random.randint(0, len(config))
    spin = config[a,b]
    nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]            
    emn = J*spin*nb
    if emn < 0:
        config[a,b] = -config[a,b]
    else:
        p = np.exp(-emn*beta)
        u = np.random.rand()
        if u < p:
            config[a,b] = -config[a,b]
    return config

In [12]:
Temp = 0.1
beta = 1./Temp
N = 20

figure = plt.figure()
grid = initialstate(N)

images = []

for i in range(3000):
    im = plt.imshow(mcmove(grid,beta,2))
    images.append([im])
    
ani = animation.ArtistAnimation(figure, images, interval=500, blit=True,
                                repeat = False)


ani.save("simpleising_slow.mp4",writer = 'ffmpeg') #saving the file!

## Simulation of model with external magnetic field

#####  Scalar Magnetic Field

In [26]:
def mcmove_ext(config,beta,J,ext):
    N = len(config)
    a = np.random.randint(0, N)
    b = np.random.randint(0, N)
    spin = config[a,b]
    nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]       
    emn = J*spin*nb - ext*spin
    if emn < 0:
        config[a,b] = -config[a,b]
    else:
        p = np.exp(-emn*beta)
        u = np.random.rand()
        if u < p:
            config[a,b] = -config[a,b]
    return config

High temperature wants to make it a random spin states, but with "high" external magnetic field, it outweighs the temperature parameter and the spins follow the field. (with some hesitant from the temperature parameter)

In [27]:
Temp = 5
beta = 1./Temp


N = 20

figure = plt.figure()
grid = initialstate(N)

images = []

for i in range(3000):
    im = plt.imshow(mcmove_ext(grid,beta,2.0,5))
    images.append([im])
    
    
    
ani = animation.ArtistAnimation(figure, images, interval=7, blit=True,
                                repeat_delay=20000)


ani.save('fast_ext_ising.mp4',writer = 'ffmpeg')

##### Vectorized Magnetic Field

In [18]:
def mcmove_vext(config,beta,J,field):
    N = len(config)
    a = np.random.randint(0, N)
    b = np.random.randint(0, N)
    field_strength = field[a,b]
    spin = config[a,b]
    nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]       
    emn = J*spin*nb - field_strength*spin
    if emn < 0:
        config[a,b] = -config[a,b]
    else:
        p = np.exp(-emn*beta)
        u = np.random.rand()
        if u < p:
            config[a,b] = -config[a,b]
    return config

In [20]:
Temp = 3
beta = 1./Temp


N = 20

ext_mag = np.ones((N,N))

ext_mag[:int(N/2.),:int(N/2.)]= 1
ext_mag[int(N/2.):,:int(N/2.)]= 20
ext_mag[int(N/2.):,int(N/2.):]= 1
ext_mag[:int(N/2.),int(N/2.):] = 20




figure = plt.figure()
grid = initialstate(N)

images = []

for i in range(3000):
    im = plt.imshow(mcmove_vext(grid,beta,2.0,ext_mag))
    images.append([im])
    
    
    
ani = animation.ArtistAnimation(figure, images, interval=500, blit=True,
                                repeat_delay=20000)


ani.save('slow_vext_ising.mp4',writer = 'ffmpeg')

## Hexagonal Ising Model

In [3]:
def hex_mcmove_ext(config, beta,J,ext):
    N = len(config)
    for i in range(len(config)):
        for j in range(len(config)):
            a = np.random.randint(0, len(config))
            b = np.random.randint(0, len(config))
            spin = config[a,b] 
            nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N] + config[(a+1)%N,(b+1)%N] + config[(a-1)%N,(b+1)%N] #new spins here       
            emn = J*spin*nb - ext*spin
            if emn < 0:
                config[a,b] = -config[a,b]
            else:
                p = np.exp(-emn*beta)
                u = np.random.rand()
                if u < p:
                    config[a,b] = -config[a,b]
    return config

In [10]:
temp = 0.05
iterations = 1000
N = 50
J = -2
hex_grid1 = initialstate(N)

for it in range(iterations):
    hex_grid1 = hex_mcmove_ext(hex_grid1,1./temp,J,3)
    
    
    
    
fig,ax=plt.subplots(1,2,figsize=(15,12),squeeze = False)


temp = 3
iterations = 1000
N = 50
J = -2
hex_grid2 = initialstate(N)

for it in range(iterations):
    hex_grid2 = hex_mcmove_ext(hex_grid2,1./temp,J,3)
    
    
    

    
ax[0][0].imshow(hex_grid1)
ax[0][0].set_title('After '+ str(iterations*iterations*iterations)+' Simulation; $J$ = -2, T = 0.05')
ax[0][1].imshow(hex_grid2)
ax[0][1].set_title('After'+str(iterations*iterations*iterations)+' Simulation; $J = -2$, T = 3')

Text(0.5,1,'After1000000000 Simulation; $J = -2$, T = 3')

In [14]:
nt      = 100        #  number of temperature points
N       = 22         #  size of the lattice, N x N
iterations = 1024       # number of iterations for convergence + calculations

T = 0.05
ext_mag = np.linspace(0,10,nt)
M = np.zeros(nt)
n1 = 1.0/(iterations*N*N)

for tt in range(len(ext_mag)):
    M1 = 0
    grid = initialstate(N)
    
    for i in range(iterations):     #loop to convergence first
        grid = hex_mcmove_ext(grid,1./T,-1,ext_mag[tt])
    
    for i in range(iterations):     #start calculating the quantities when converged
        grid = hex_mcmove_ext(grid,1./T,-1,ext_mag[tt])
        mag = get_Mag(grid)
        
        M1 = M1 + mag

    M[tt] = n1*M1

In [17]:
plt.figure(figsize=(12,15))
plt.plot(ext_mag,abs(M),'o')
plt.xlabel('Magnitude of Magnetic Field',fontsize=12)
plt.ylabel('Magnetization',fontsize = 12)
plt.grid()


In [18]:
T = 0.05
extM = 3.5

grid_x = initialstate(50)

for i in range(iterations*2):
    grid_x = hex_mcmove_ext(grid_x,1./T,-1,extM)
    
plt.figure(figsize=(12,15))
plt.imshow(grid_x)

In [24]:
N = 20

figure = plt.figure()
grid_s = initialstate(N)

images = []

for i in range(3000):
    im = plt.imshow(hex_mcmove_ext(grid_s,1./T,-1,extM))
    images.append([im])
    
    
    
ani = animation.ArtistAnimation(figure, images, interval=200, blit=True,
                                repeat_delay=20000)


ani.save('hex.mp4',writer = 'ffmpeg')